## Import Required Libraries

In [1]:
import json
import time
import os
import csv
import json
import numpy as np
import pandas as pd
import random
from itertools import combinations
import math
import sys
import pickle
import spotipy_api
import random

## Load information about Playlist and Tracks

In [2]:
t0 = time.time()
# Load data matching playlist ID to Track ID
df_play_track = pd.read_csv('./data/track_w_playlist_id.csv')

# Load file containing the Track names
fr = open('./data/tracks_w_names.csv', 'r', encoding="utf-8")
reader = csv.reader(fr)
track_name = []
for row in reader:
    track_name.append(row[1])
fr.close()

In [3]:
# Load file containing candidate pair 
fr = open("./data/candidate_pair_1rows_10bands.pickle", "rb")
can_pairs = pickle.load(fr, encoding='bytes')
fr.close()

# Sort data to make it easier to search
can_pairs = can_pairs[can_pairs[:, 0].argsort()]

## Define Function to Recommend Tracks

In [4]:
def song_recommendation(playlist_track_ids, k, candidate_pairs, track_with_name):
    # Get all similar tracks using candidate pairs
    track_ids = np.array([], dtype=int)
    for i in playlist_track_ids:
        idx_start = np.searchsorted(candidate_pairs[:, 0], i, side='left')
        idx_end = np.searchsorted(candidate_pairs[:, 0], i, side='right')
        can_track_ids = candidate_pairs[np.array(range(idx_start,idx_end+1)),1]
        track_ids = np.append(track_ids, can_track_ids)
    
    # Get count of number of each recommended track
    track_rec_id_count = dict([])
    for track in track_ids:
        count = track_rec_id_count.get(track, 0)
        track_rec_id_count[track] = count + 1
    
    # Convert track dictionary to list for easier use
    track_vs_count = []
    for key, value in track_rec_id_count.items():
        track_vs_count.append([value, key])
    
    # Sort track recommendation by suggestion count
    track_vs_count.sort(reverse=True)
    track_vs_count = np.array(track_vs_count)
    
    # Return top K results
    final_ids = track_vs_count[0:k,1]
    track_names = []
    for ele in final_ids:
        track_names.append(track_with_name[ele])
    return (track_names, final_ids)

## Playlist Addition Recommedation 

In [5]:
# Get list of tracks from a specific playlist
playlist_track_list = df_play_track[df_play_track['Playlist ID'] == 0]["Track ID"].to_list()

# Get number of input and test songs from playlist
num_input_songs = round(len(playlist_track_list)*0.8)
num_test_songs = len(playlist_track_list)-num_input_songs

# Get random set of input songs
random.shuffle(playlist_track_list)
input_songs = []
for i in range(0,num_input_songs):
    input_songs.append(playlist_track_list.pop())
    
# Get random set of test songs
test_songs = [] 
for i in range(0,num_test_songs):
    test_songs.append(playlist_track_list.pop())

# Get song recommendations
recom_songs = song_recommendation(input_songs, num_test_songs, can_pairs, track_name)[0]

In [6]:
# Display recommendation songs
print('Recommendations:')
for i in range(0,num_test_songs):
    print(recom_songs[i])
    
# Display test songs
print('\nActual Songs:')
for i in range(0,num_test_songs):
    print(track_name[test_songs[i]])

Recommendations:
Sk8er Boi - Avril Lavigne
My First Kiss - feat. Ke$ha - 3OH!3
Promiscuous - Nelly Furtado
Eenie Meenie - Justin Bieber
Beautiful Girls - Sean Kingston
Right Now (Na Na Na) - Akon
Fergalicious - Fergie
Apologize - OneRepublic
Down - Jay Sean
The Sweet Escape - Gwen Stefani

Actual Songs:
Closer - Ne-Yo
Dirty Little Secret - The All-American Rejects
Yeah! - Usher
Me & U - Cassie
Lose Control (feat. Ciara & Fat Man Scoop) - Missy Elliott
See You Again - Miley Cyrus
Replay - Iyaz
Livin on Sunday - Cris Cab
Hey Ya! - Radio Mix / Club Mix - OutKast
A Thousand Miles - Vanessa Carlton


In [20]:
# Spotipy API
sp_api = spotipy_api.EasySpotipy()

In [46]:
PLAYLIST_SIZE=30
INPUT_SIZE=10
NUM_SONGS_TO_RECOMMEND=20
TOTAL_SONGS_TO_PRINT=max(INPUT_SIZE + NUM_SONGS_TO_RECOMMEND, PLAYLIST_SIZE)

random_playlist = sp_api.getRandomPlaylist(min_tracks=PLAYLIST_SIZE, max_tracks=PLAYLIST_SIZE)
random_playlist_indices = random_playlist[:,1].tolist()
print(f'Random playlist:\n{random_playlist[:,2]}')

Random playlist:
['Do You Want to Build a Snowman? - From "Frozen"/Soundtrack Version - Kristen Bell, Agatha Lee Monn, Katie Lopez'
 'Let It Go - From "Frozen"/Soundtrack Version - Idina Menzel'
 'Love Is an Open Door - From "Frozen"/Soundtrack Version - Kristen Bell, Santino Fontana'
 'One Jump Ahead - Brad Kane, Disney'
 'A Whole New World - Lea Salonga, Brad Kane, Disney'
 'For the First Time in Forever - From "Frozen"/Soundtrack Version - Kristen Bell, Idina Menzel'
 'Arabian Nights - Bruce Adler, Disney' "You're Welcome - Dwayne Johnson"
 'Mine, Mine, Mine - From "Pocahontas"/Soundtrack Version - David Ogden Stiers, Mel Gibson'
 'You\'ve Got a Friend in Me - From "Toy Story"/Soundtrack Version - Randy Newman'
 'On My Way - From "Brother Bear"/Soundtrack Version - Phil Collins'
 'Hawaiian Roller Coaster Ride - From "Lilo & Stitch"/Soundtrack Version - Mark Keali\'i Ho\'omalu, Kamehameha Schools Children\'s Chorus'
 'Honor To Us All - Lea Salonga, Beth Fowler, Marni Nixon, Chorus - 

In [47]:
# Randomize songs within playlist
random.shuffle(random_playlist_indices)

(recom_song_names, recom_song_indices) = song_recommendation(random_playlist_indices[0:INPUT_SIZE], NUM_SONGS_TO_RECOMMEND, can_pairs, track_name)

In [52]:
random_playlist_names = []
for index in random_playlist_indices:
    random_playlist_names.append(track_name[index])

print('{:<80}{:^10}{:<80}'.format('Original', '----', 'Recommended'))
print('--------------------------------------------------------------------------------------')
for i in range(TOTAL_SONGS_TO_PRINT):
    if i <= INPUT_SIZE:
        print('{:<80}{:^10}{:<80}'.format(track_name[random_playlist_indices[i]], '----', track_name[random_playlist_indices[i]]))
    elif i > INPUT_SIZE and i < PLAYLIST_SIZE:
        print('{:<80}{:^10}{:<80}'.format(track_name[random_playlist_indices[i]], '----', recom_song_names[i - INPUT_SIZE]))
    elif i > PLAYLIST_SIZE:
        print('{:<80}{:^10}{:<80}'.format('', '----', recom_song_names[i - INPUT_SIZE]))
    if i == INPUT_SIZE:
        print(f'\nInputs above -----------------------------------------------------------\n')

print('Songs in both: ')
shared_songs = list(set(recom_song_names).intersection(random_playlist_names))
for song in shared_songs:
    print(song)

Original                                                                           ----   Recommended                                                                     
--------------------------------------------------------------------------------------
Honor To Us All - From "Mulan"/Soundtrack - Beth Fowler                            ----   Honor To Us All - From "Mulan"/Soundtrack - Beth Fowler                         
Beauty and the Beast - Angela Lansbury                                             ----   Beauty and the Beast - Angela Lansbury                                          
Under the Sea - From "The Little Mermaid"/ Soundtrack Version - Samuel E. Wright   ----   Under the Sea - From "The Little Mermaid"/ Soundtrack Version - Samuel E. Wright
One Jump Ahead - Brad Kane                                                         ----   One Jump Ahead - Brad Kane                                                      
You're Welcome - Dwayne Johnson                           

In [39]:
# Create a new playlist
original_playlist = sp_api.createNewPlaylist(random_playlist[:,0], 'Original Playlist')

# Create playlist with recommendations instead
recom_song_ids = [sp_api.getTrackIDFromIndex(idx) for idx in recom_song_indices]
new_playlist   = sp_api.createNewPlaylist(recom_song_ids + random_playlist[0:INPUT_SIZE,0].tolist(), 'New Playlist!')